In [ ]:
# Library Import
import pandas as pd

# Data Import
df=pd.read_csv('VAT_Dataset.csv', sep=";")

# Change column names to a standard
df.rename(columns={'ID_x': 'Customer_ID', 'Country_x': 'Country','VAT': 'Check_Column'}, inplace=True)

# Show top 5 rows
df.head()


In [ ]:
df.shape

In [ ]:
df.loc[(df['Check_Column'].isnull())]

In [ ]:
# List missing value
df.loc[(df['Check_Column'].isnull())]

# Drop rows where Check_Column is missing
df=df.dropna(subset=['Check_Column'])

# Reset index for dropped rows
df.reset_index(inplace=True, drop=True)

# The new size of df
df.shape

In [ ]:
# Group by country
dfg = df.groupby(['Country']).size()

# Plot
dfg.plot(kind='bar', ylabel='Customer Count',
         xlabel='Country', figsize=(10, 6))

# Section 1 - Pattern Frequency

### List all characters used in the column

In [ ]:
# 1. String conversion
df['Check_Column']=df['Check_Column'].map(str)

# 2. Split Check_Column
df['Check_Column_Split']=df['Check_Column'].apply(list)

# Show top 5 records
df.head()

## Overview of all characters (Optional Section)

In [ ]:
lst = list(range(1,10000))
lst_1=[chr(item) for item in lst]

In [ ]:
df_all_ascii = pd.DataFrame()
df_all_ascii['Ord']  = lst
df_all_ascii['Chr']  = lst_1
df_all_ascii['Type']  = df_all_ascii['Chr'].apply(type)
print(df_all_ascii.loc[1200:1300])

## Ascii code decoder
Ascii codes<br>
<48 Chr_1<br>
<58 Num<br>
<65 Chr_2<br>
<91 Let<br>
<97 Chr_3<br>
<123 Let<br>
Chr_4

In [ ]:
print("ASCII code of Ä: " + str(ord("Ä")))
print("ASCII code of Ç: " + str(ord("Ç")))
print("ASCII code of ä: " + str(ord("ä")))
print("ASCII code of ç: " + str(ord("ç")))

In [ ]:
# Define a list for additional letters
add_L_arr = ['Ä', 'ä', 'Ç', 'ç']

# Translate additional letters to ASCII
add_L_arr_ord = list(map(ord, add_L_arr))

# Show
add_L_arr_ord

In [ ]:
# Create function
def ascii_translator(text_list):
    result=[]
    for x in text_list:
        x=ord(x) # Converting characters to ASCII
        if x in (add_L_arr_ord): # Condition for additional letter ASCII representations
            result.append("L") # Letter
        elif 32<=x<=47 or 58<=x<=64 or 91<=x<=96 or 123<=x:
            result.append("S") # Special Characters
        elif 65<=x<=90 or 97<=x<=122:
            result.append("L") # Letter
        elif 48<=x<=57:
            result.append("N") # Numeric
        else:
            result.append("U") # Uncategorized
    return '-'.join(result)

# Apply function to df
df['Check_Column_Translated']=df['Check_Column_Split'].apply(ascii_translator)

# Show top 5 rows
df.head()

In [ ]:
# 1. Number of patterns by country
dfg_1 = df.groupby(['Country','Check_Column_Translated']).size().to_frame(name = 'Pattern_Count').reset_index()

# 2. Number of total customers by country
dfg_2 = df.groupby(['Country']).size().to_frame(name = 'Country_Subtotal').reset_index()

# 3. Merge both grouped tables
grp_tbl = pd.merge(dfg_1, dfg_2, how='inner', on=['Country'])

# 4. Pattern percentage calculation
grp_tbl['Pattern_Pct']=grp_tbl['Pattern_Count']/grp_tbl['Country_Subtotal']

# Show top rows
grp_tbl.head()

In [ ]:
# Filter and remove unusued columns
dfg=grp_tbl[grp_tbl['Country']=="AA"][['Check_Column_Translated','Pattern_Pct']]

# Plot
dfg.plot(x='Check_Column_Translated',y='Pattern_Pct',kind='bar', ylabel='Pattern Pct',
         xlabel='Pattern', figsize=(10, 6))

In [ ]:
dfg

In [ ]:
# Group by country max pattern pct (We keep Check_Column_Translated to be used in Reporting section)
grp_cntry_max=grp_tbl.loc[grp_tbl.groupby('Country')['Pattern_Pct'].idxmax()][['Country','Check_Column_Translated','Pattern_Pct']]

# Plot
grp_cntry_max[['Country','Pattern_Pct']].plot(x='Country',y='Pattern_Pct',kind='bar', ylabel='Pattern Pct',
         xlabel='Country', figsize=(10, 6))

In [ ]:
# Merge customer level table with country-pattern level table
df_all_temp = pd.merge(df, grp_tbl, how='inner', on=['Country','Check_Column_Translated'])

# Remove unused column
df_all_temp.drop('Check_Column_Split',1,inplace=True)

# Show table
df_all_temp

In [ ]:
# Assign threshold
threshold=0.01

# Filter customer that are under threshold
df_all = df_all_temp[df_all_temp.Pattern_Pct<=threshold].reset_index()

# Show df_all
df_all

In [ ]:
grp_cntry_max.head()

In [ ]:
# Renaming df_all_temp table. C is used for 'Customer', MC is used for 'Most Common'
df_all.rename(columns={'Check_Column_Translated': 'C_Pattern', 
                            'Pattern_Count': 'C_Pttrn_Count',
                            'Country_Subtotal': 'C_No_by_Cntry',
                            'Pattern_Pct': 'C_Pttrn_Pct'}, inplace=True)

# Renaming grp_cntry_max table. MC is used for 'Most Common'
grp_cntry_max.rename(columns={'Check_Column_Translated': 'MC_Pattern', 
                              'Pattern_Pct': 'MC_Pttrn_Pct'}, inplace=True)

# Merge customer level table with country most common pattern table
df_report = pd.merge(df_all, grp_cntry_max, how='inner', on=['Country'])

# Remove unused column
df_report.drop('index',axis=1,inplace=True)

# Show df_report
df_report.head()

In [ ]:
# Groupby for an exception count per country
df_vs=df_report.groupby(['Country','C_No_by_Cntry'],as_index=False)['Customer_ID'].count()
df_vs.rename(columns={'Customer_ID':'exc_count'}, inplace=True)

# Sort according to exception count
df_vs.sort_values('exc_count', ascending=False, inplace=True)

# Plot exception count for top 10 countries 
df_vs.head(10).plot(kind='pie',y='exc_count',labels=df_vs['Country'], legend = False, figsize=(16,8), autopct='%1.f%%')

In [ ]:
# Calculate exception percentage
df_vs['Exception_Pct']=df_vs['exc_count']/df_vs['C_No_by_Cntry']

# Sort according to exception percentage
df_vs.sort_values('Exception_Pct', ascending=False, inplace=True)

# Plot
df_vs.plot(x='Country',y='Exception_Pct',kind='bar',
         xlabel='Country', figsize=(10, 6))